In [1]:
import geopandas as gpd
import os
os.sys.path.append("../")
from scripts.geospatial_analysis import *
from pyspark.sql import functions as F, SparkSession

In [2]:
# Create a Spark Session
spark = (
    SparkSession.builder.appName("geospatial analysis")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.execturo.memory", "2g")
    .getOrCreate()
)

24/09/07 16:52:41 WARN Utils: Your hostname, qinsitaodeMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.4.51 instead (on interface en0)
24/09/07 16:52:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/07 16:52:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
consumer_info = spark.read.parquet('../data/curated/consumer_info.parquet')
fraudulent_consumer_rate = spark.read.parquet('../data/curated/consumer_fraud_rate.parquet')

In [12]:

fraudulent_consumer_with_info = consumer_info.join(fraudulent_consumer_rate, on="consumer_id", how="inner")

In [10]:
consumer_transaction_records = transaction_records.join(consumer_info, on="consumer_id", how="inner")

consumer_id,merchant_abn,dollar_value,order_id,order_datetime,name,gender,state,postcode
1059280,79417999332,136.06570809815838,23acbb7b-cf98-458...,2021-11-26,Cameron Adams,Male,QLD,4563
1195503,46451548968,72.61581642788431,76bab304-fa2d-400...,2021-11-26,Yolanda Williams,Female,WA,6935
986886,89518629617,3.0783487174439297,a2ae446a-2959-41c...,2021-11-26,Maria Riley,Female,SA,5157
1195503,49167531725,51.58228625503599,7080c274-17f7-4cc...,2021-11-26,Yolanda Williams,Female,WA,6935
986886,31101120643,25.228114942417797,8e301c0f-06ab-45c...,2021-11-26,Maria Riley,Female,SA,5157
179208,67978471888,691.5028234458998,0380e9ad-b0e8-420...,2021-11-26,Mary Smith,Female,NSW,2782
986886,60956456424,102.13952056640888,5ac3da9c-5147-452...,2021-11-26,Maria Riley,Female,SA,5157
179208,47644196714,644.5220654863093,4e368e44-86f8-4de...,2021-11-26,Mary Smith,Female,NSW,2782
267457,39649557865,209.12780951421405,4d78cd01-4bab-494...,2021-11-26,Jasmine Ford,Undisclosed,NSW,2625
1194530,88402174457,141.0387993699113,c50c957d-ecfc-430...,2021-11-26,Jill Jones MD,Female,NT,862


In [ ]:
# Find info for fraudulent consumer
fraudulent_consumer_with_info = consumer_info.join(fraudulent_consumer_rate, on="consumer_id", how="inner")

In [ ]:
# Number of consumer in each state
consumer_info.groupby("state").agg(F.count("consumer_id"))

In [ ]:
# Number of fraudulent consumer in each state
fraudulent_consumer_with_info.groupby("state").agg(F.count("consumer_id"))

In [ ]:
fraudulent_consumer_with_info.count()
#consumer_info.count()
#fraudulent_consumer_with_info.select("consumer_id").distinct().count() # Number of unique fraudulent consumer

In [ ]:
# Check the number of NULL values
null_counts = fraudulent_consumer_with_info.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in consumer_fraud_with_info.columns])
null_counts.limit(1)
# null_counts = consumer_info.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in consumer_info.columns])
# null_counts.limit(1)


In [ ]:
# Number of consumer in each postcode
consumer_info.groupBy(["postcode"]).agg(F.count("consumer_id").alias("total_consumer"))

# External dataset

ABS Digital Boundary files

`States and Territories - 2021 - Shapefile`

https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files#downloads-for-gda2020-digital-boundary-files

`Australian postcodes - Shapefile`

https://www.matthewproctor.com/australian_postcodes


In [ ]:
# Transform the full state name to the abbriviation name
state_abbr = {
    "New South Wales": "NSW",
    "Victoria": "VIC",
    "Queensland": "QLD",
    "South Australia": "SA",
    "Western Australia": "WA",
    "Tasmania": "TAS",
    "Northern Territory": "NT",
    "Australian Capital Territory": "ACT"
}

In [ ]:
# Load the shapefile into a GeoDataFrame
postcode_shapefile_path = '../data/raw/POA_2021_AUST_GDA2020_SHP/POA_2021_AUST_GDA2020.shp'
gdf_postcode = gpd.read_file(postcode_shapefile_path)

state_shapefile_path = '../data/raw/STE_2021_AUST_SHP_GDA2020/STE_2021_AUST_GDA2020.shp'
gdf_state = gpd.read_file(state_shapefile_path)
gdf_state['STE_NAME21'] = gdf_state['STE_NAME21'].replace(state_abbr)

In [ ]:
fraudulent_consumer_with_info

In [ ]:
# Number of (fraudulent) consumer in each postcode or state
consumer_group_by_postcode = consumer_info.groupBy(["postcode"]).agg(F.count("consumer_id").alias("total_consumer"))
consumer_group_by_state = consumer_info.groupBy(["state"]).agg(F.count("consumer_id").alias("total_consumer"))

fraudulent_consumer_group_by_postcode = fraudulent_consumer_with_info.groupBy(["postcode"]).agg(F.count("consumer_id").alias("total_consumer"),
                                                                                                 F.avg("fraud_probability").alias("average_fraud_prob"),
                                                                                                 F.max("fraud_probability").alias("max_fraud_probability"))
fraudulent_consumer_group_by_state = fraudulent_consumer_with_info.groupBy(["state"]).agg(F.count("consumer_id").alias("total_consumer"),
                                                                                                 F.avg("fraud_probability").alias("average_fraud_prob"),
                                                                                                 F.max("fraud_probability").alias("max_fraud_probability"))

In [ ]:
# Group the consumer and fraudulent consumer by different features
consumer_group_by_postcode = consumer_group_by_postcode.toPandas()
consumer_group_by_state = consumer_group_by_state.toPandas()

fraudulent_consumer_group_by_postcode = fraudulent_consumer_group_by_postcode.toPandas()
fraudulent_consumer_group_by_state = fraudulent_consumer_group_by_state.toPandas()

consumer_group_by_postcode['postcode'] = consumer_group_by_postcode['postcode'].astype(str)
consumer_group_by_state['state'] = consumer_group_by_state['state'].astype(str)

fraudulent_consumer_group_by_postcode['postcode'] = fraudulent_consumer_group_by_postcode['postcode'].astype(str)
fraudulent_consumer_group_by_state['state'] = fraudulent_consumer_group_by_state['state'].astype(str)

gdf_postcode['POA_CODE21'] = gdf_postcode['POA_CODE21'].astype(str)

In [ ]:
# Merge consumer information with geographic data from the shapefile
total_consumer_group_by_postcode_merged_gdf = gdf_postcode.merge(consumer_group_by_postcode, left_on='POA_CODE21', right_on='postcode')
total_consumer_group_by_postcode__geojson = total_consumer_group_by_postcode_merged_gdf.to_json()

fraudulent_consumer_group_by_postcode_merged_gdf = gdf_postcode.merge(fraudulent_consumer_group_by_postcode, left_on='POA_CODE21', right_on='postcode')
fraudulent_consumer_group_by_postcode_geojson = fraudulent_consumer_group_by_postcode_merged_gdf.to_json()



In [ ]:
# Merge consumer information with geographic data from the shapefile
total_consumer_group_by_state_merged_gdf = gdf_state.merge(consumer_group_by_state, left_on='STE_NAME21', right_on='state')
total_consumer_group_by_state__geojson = total_consumer_group_by_state_merged_gdf.to_json()

fraudulent_consumer_group_by_state_merged_gdf = gdf_state.merge(fraudulent_consumer_group_by_state, left_on='STE_NAME21', right_on='state')
fraudulent_consumer_group_by_state_geojson = fraudulent_consumer_group_by_state_merged_gdf.to_json()



In [ ]:
# Distribution of number of customer group by postcode
# Computer might be lagging, better to clear output of this cell
create_consumer_map(total_consumer_group_by_postcode__geojson,consumer_group_by_postcode, "postcode", "total_consumer", "POA_CODE21")

In [ ]:
# Distribution of average fraud probability group by postcode
create_consumer_map(fraudulent_consumer_group_by_postcode_geojson,fraudulent_consumer_group_by_postcode, "postcode", "average_fraud_prob", "POA_CODE21")

In [ ]:
# Distribution of maximum fraud probability group by postcode
create_consumer_map(fraudulent_consumer_group_by_postcode_geojson,fraudulent_consumer_group_by_postcode, "postcode", "max_fraud_probability", "POA_CODE21")

In [ ]:
# Group by state instead
create_consumer_map(total_consumer_group_by_state__geojson,consumer_group_by_state, "state", "total_consumer","STE_NAME21")

In [ ]:
create_consumer_map(fraudulent_consumer_group_by_state_geojson,fraudulent_consumer_group_by_state, "state", "average_fraud_prob","STE_NAME21")

In [ ]:
create_consumer_map(fraudulent_consumer_group_by_state_geojson,fraudulent_consumer_group_by_state, "state", "max_fraud_probability","STE_NAME21")